In [ ]:
!pip install neptune-client

In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import neptune.new as neptune

from sklearn.model_selection import train_test_split

from tensorflow import keras

RANDOM_STATE = 126

# Network framework

In [ ]:
# model json structure
_description = "description"
_layers = "layers"
_name = "name"
_params = "params"
_optimizer = "optimizer"
_loss = "loss"
_metrics = "metrics"

# nn layers
_Dense = "Dense"
_Dropout = "Dropout"
_Conv2d = "Conv2d"
_BatchNormalization = "BatchNormalization"
_MaxPooling2D = "MaxPooling2D"
_Flatten = "Flatten"
_BatchNormalization = "BatchNormalization"
_Dropout = "Dropout"
_GlobalAveragePooling2D = "GlobalAveragePooling2D"

# nn layers params
_units = "units"
_activation = "activation"
_rate = "rate"
_filters = "filters"
_kernel_size = "kernel_size"
_strides = "strides"
_padding = "padding"
_input_shape = "input_shape"
_pool_size = "pool_size"
_rate = "rate"

# activation functions
_relu = "relu"
_softmax = "softmax"

# optimizers
_SGD = "SGD"
_Adam = "Adam"
_RMSprop = "RMSprop"

# optimizers params
_learning_rate = "learning_rate"
_momentum = "momentum"
_rho = "rho"

# losses
_CategoricalCrossentropy = "CategoricalCrossentropy"
_from_logits = "from_logits"

# metrics
_accuracy = "accuracy"

# fitting params
_epochs = "epochs"
_batch_size = "batch_size"


In [ ]:
nn_layers = {
    _Dense: keras.layers.Dense,
    _Dropout: keras.layers.Dropout,
    _Conv2d: keras.layers.Conv2D,
    _BatchNormalization: keras.layers.BatchNormalization,
    _MaxPooling2D: keras.layers.MaxPooling2D,
    _Flatten: keras.layers.Flatten,
    _BatchNormalization: keras.layers.BatchNormalization,
    _Dropout: keras.layers.Dropout,
    _GlobalAveragePooling2D: keras.layers.GlobalAveragePooling2D
}

nn_optimizers = {
    _SGD: keras.optimizers.SGD,
    _Adam: keras.optimizers.Adam,
    _RMSprop: keras.optimizers.RMSprop,
}

nn_losses = {
    _CategoricalCrossentropy: keras.losses.CategoricalCrossentropy,
}

def make_nn(run_json, run=None):
    """
    Создание модели нейросети из json описания эксперимента
    """
    # выделяем слои сети и создаём пустой массив для объектов слоёв
    model_layers_json = run_json[_layers]
    model_layers = []
    
    model_layers.append(keras.layers.Lambda(lambda x: x/255))

    # поочерёдно добавляем слои в сеть
    for layer_json in model_layers_json:
        if _name in layer_json and _params in layer_json:
            layer_type = nn_layers[layer_json[_name]]
            layer_params = layer_json[_params]

            layer = layer_type(**layer_params)
            model_layers.append(layer)

    # создаём объект НС
    model = keras.Sequential(model_layers)

    # записываем параметры
    if run is not None:
        run["model"] = model_layers_json

    return model


def make_optimizer(run_json, run=None):
    """
    Создание модели оптимизатора из json описания эксперимента
    """
    optimizer_json = run_json[_optimizer]
    optimizer_type = nn_optimizers[optimizer_json[_name]]
    optimizer_params = optimizer_json[_params]

    optimizer = optimizer_type(**optimizer_params)

    if run is not None:
        run["optimizer"] = optimizer_json[_name]
        run["learning_rate"] = optimizer_params[_learning_rate]

    return optimizer


def make_loss(run_json, run=None):
    """
    Создание объекта функции потерь
    """
    loss = nn_losses[run_json[_loss]]()

    if run is not None:
        run["loss"] = _loss

    return loss


def compile_nn(run_json, model, optimizer, loss, run=None):
    """
    Компиляция модели
    """
    model.compile(loss=loss, optimizer=optimizer, metrics=[run_json[_metrics]])

    if run is not None:
        run["metrics"] = run_json[_metrics]


def fit_nn(run_json, model, x_train, y_train, run=None, lr_schedule_on=False):
    """
    Обучение модели
    """
    epochs = run_json[_epochs]
    batch_size = run_json[_batch_size]

    if run is not None:
        run["epochs"] = epochs
        run["batch_size"] = batch_size
        
    if lr_schedule_on:
        optimizer_lr = run_json[_optimizer][_params][_learning_rate]
        lr_schedule = keras.callbacks.LearningRateScheduler(
            lambda epoch: optimizer_lr / 10**(epoch / 20))
        callbacks = [lr_schedule]
    else:
        callbacks = []

    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks)

    if run is not None:
        run["train/loss"].log(history.history["loss"])

        metrics = run_json[_metrics]
        for metric in metrics:
            run["train/" + metric].log(history.history[metric])
            
    return history

def evalueate_nn(model, x_test, y_test, run=None):
    eval_accuracy = model.evaluate(x_test, y_test)[1]
    
    if run is not None:
        run["eval/accuracy"] = eval_accuracy
        
    return eval_accuracy


In [ ]:
def Dense(units, activation=_relu):
    return {
      _name: _Dense,
      _params: {
          _units: units,
          _activation: activation
      }
    }

def Conv(filters, kernel_size):
    return {
      _name: _Conv2d,
      _params: {
          _filters: filters,
          _kernel_size: kernel_size,
          _activation: _relu
      }
    }

def MaxPooling(pool_size):
    return {
      _name: _MaxPooling2D,
      _params: {
          _pool_size: pool_size
      }
    }

def AvgPooling():
    return {
        _name: _GlobalAveragePooling2D,
        _params: {}
    }



def Flatten():
    return {
        _name: _Flatten,
        _params: {}
    }

def BatchNormalization():
    return {
        _name: _BatchNormalization,
        _params: {}
    }

def Dropout(rate):
     return {
        _name: _Dropout,
        _params: {
            _rate: rate
        }
    }


def SGD(lr, momentum=0.9):
    return {
      _name: _SGD,
      _params: {
          _learning_rate: lr,
          _momentum: momentum
      }
    }

def Adam(lr):
    return {
      _name: _Adam,
      _params: {
          _learning_rate: lr
      }
    }

def RMSProp(lr):
    return {
      _name: _RMSprop,
      _params: {
          _learning_rate: lr
      }
    }

# Data preparing

In [ ]:
data_root_path = "../input/cassava-leaf-disease-classification"
train_path = data_root_path + "/train_images/"
train_df_path = data_root_path + "/train.csv"

## Selecting images to train and evaluate

In [ ]:
train_df = pd.read_csv(train_df_path)

data_len = len(train_df)
classes_num = len(set(train_df["label"]))

print(f"Data lenght: {data_len}")
print(f"Classes: {classes_num}")

train_df.head()

In [ ]:
total_one_class_obj_num = 600
imgs_info = list()

for class_label in set(train_df["label"]):
    class_data = train_df.loc[train_df["label"] == class_label]
    
    sampled_class_data = class_data.sample(n=total_one_class_obj_num, random_state=RANDOM_STATE)
    imgs_info.extend(list(zip(sampled_class_data["image_id"].values, sampled_class_data["label"].values)))

print(f"Images sampled: {len(imgs_info)}")

## Reading selected images

In [ ]:
def read_img(img_name):
    img_bgr = cv2.imread(train_path + img_name)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    return img_rgb

imgs = [ (read_img(im_name), im_class) for im_name, im_class in imgs_info ]

images_shape = np.mean([ img.shape for img, _ in imgs ], axis=0).astype(int)

print(f"Images read: {len(imgs)}")
print(f"Images shape: {images_shape}")

## Resizing

In [ ]:
target_size = (256, 256)
imgs_reszd = [ (cv2.resize(im, target_size), im_class) for im, im_class in imgs ]

ax = plt.subplot(121)
ax.imshow(imgs[0][0])
ax.set_title("Original")

ax = plt.subplot(122)
ax.imshow(imgs_reszd[0][0])
ax.set_title("Resized");

## Data splitting

In [ ]:
x = []
y = []

for img_data, img_class in imgs_reszd:
    x.append(img_data)
    
    # one-hot encoding for y
    one_hot_y = np.zeros(classes_num)
    one_hot_y[img_class] = 1
    
    y.append(one_hot_y)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    test_size=200 * classes_num,
    stratify=y,
    random_state=RANDOM_STATE
)

x_train = np.array(x_train, dtype=np.float32 )
x_test = np.array(x_test, dtype=np.float32 )
y_train = np.array(y_train, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32 )

print(f"Train size: {len(x_train)}")
print(f"Test size: {len(x_test)}")

# Model fitting

## Simple CNN

In [ ]:
import gc

def clean_memory():
    model = None
    optimizer = None
    history = None
    gc.collect()

In [ ]:
# просто заглушка, чтобы работало
class Run:
    history = None
    
    def __init__(self):
        history = None
    
    def __setitem__(self, key, value):
        self.__dict__[key] = value
    
    def __getitem__(self, key):
        return Run()
    
    def log(self, values):
        self.history = values
        
    def __enter__(self):
        return Run()
    
    def __exit__(self, exception_type, exception_value, traceback):
        ...

In [ ]:
if True:
    with Run() as run:
        clean_memory()

        run_json = {
          _description: "Avg poling with dropouts",
          _layers: [
            Conv(32, (3, 3)),
            BatchNormalization(),
            MaxPooling((2, 2)),
            Conv(64, (3, 3)),
            BatchNormalization(),
            MaxPooling((2, 2)),
            Conv(128, (3, 3)),
            BatchNormalization(),
            MaxPooling((2, 2)),
            AvgPooling(),
            Flatten(),
            Dense(classes_num, activation="softmax")
          ],
          _optimizer: Adam(5e-4),
          _loss: _CategoricalCrossentropy,
          _metrics: [_accuracy],
          _epochs: 50,
          _batch_size: 128

        }

        run["description"] = run_json[_description]

        # создаём модель
        model = make_nn(run_json, run)

        # создаём оптимизатор
        optimizer = make_optimizer(run_json, run)

        # инициализируем функцию потерь
        loss = make_loss(run_json, run)

        # компилируем модель
        compile_nn(run_json, model, optimizer, loss, run)

        # тренируем модель
        history = fit_nn(run_json, model, x_train, y_train, run, lr_schedule_on=False)

        # проверяем модель
        evalueate_nn(model, x_test, y_test, run)

## Transfer learning

In [ ]:
if True:
    with neptune.init(
        project="dimyakovenko/CompVision-lab6",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYjNjMzBmYS0yNDM3LTRmMTctOGI0My0xNzVkZWM2ZTQ5MTAifQ==",
    ) as run:
        clean_memory()
        
        fitted_model = keras.applications.ResNet50(weights='imagenet', 
                                 input_shape=(256, 256, 3),
                                 include_top=False)

        fitted_model.trainable = False

        input_layer = keras.Input(shape=(256, 256, 3))
        fitted_layer = fitted_model(input_layer, training=False)

        run_json = {
          _description: "ResNet50 transfer learning",
          _layers: [
            AvgPooling(),
            Dense(100),
            Dense(20),
            Dense(classes_num, activation="softmax")
          ],
          _optimizer: RMSProp(1e-2),
          _loss: _CategoricalCrossentropy,
          _metrics: [_accuracy],
          _epochs: 50,
          _batch_size: 128

        }

        run["description"] = run_json[_description]

        # создаём модель
        ceil_model = make_nn(run_json, run)(fitted_layer)
        model = keras.Model(input_layer, ceil_model)

        # создаём оптимизатор
        optimizer = make_optimizer(run_json, run)

        # инициализируем функцию потерь
        loss = make_loss(run_json, run)

        # компилируем модель
        compile_nn(run_json, model, optimizer, loss, run)

        # тренируем модель
        history = fit_nn(run_json, model, x_train, y_train, run, lr_schedule_on=False)

        # проверяем модель
        evalueate_nn(model, x_test, y_test, run)